In [1]:
import torch

a= torch.dot(torch.tensor([2,3]),torch.tensor([2,1]))
torch.bmm(torch.FloatTensor(10,3,5),torch.FloatTensor(10,5,3)).size()

torch.Size([10, 3, 3])

In [2]:
import numpy as np
data=np.load("../data/cats/fid_files/cats_64.npz")

In [23]:
data["m"].shape

(2048,)

In [3]:
import torch
import math
import numpy as np
# num_step = 4, 배치 = 3, 크기는 = 8*8
def normalize_vecs(vectors: torch.Tensor) -> torch.Tensor:
    """
    Normalize vector lengths.
    """
    return vectors / (torch.norm(vectors, dim=-1, keepdim=True))

x, y = torch.meshgrid(torch.linspace(-1, 1, 8, ),
                          torch.linspace(-1, 1, 8))
x=x.T.flatten()
y=y.T.flatten()
z = -torch.ones_like(x) / np.tan((2 * math.pi * 30 / 360)/2)

ray_d_cam= normalize_vecs(torch.stack([x, y, z], -1))
z_vals=torch.linspace(2, 6, 4).reshape(1, 4, 1).repeat(64, 1, 1)
points = ray_d_cam.unsqueeze(1).repeat(1, 4, 1) * z_vals
points = torch.stack(3*[points]) 
z_vals = torch.stack(3*[z_vals]) 
ray_d_cam = torch.stack(3*[ray_d_cam]) 

/opt/anaconda3/envs/jaepoong/lib/python3.9/site-packages/torch/functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /Users/runner/work/_temp/anaconda/conda-bld/pytorch_1656313733050/work/aten/src/ATen/native/TensorShape.cpp:2890.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [4]:
def transform_sampled_points(points, z_vals, ray_directions, h_stddev=1, v_stddev=1, h_mean=math.pi * 0.5, v_mean=math.pi * 0.5, mode='normal'):
    """Samples a camera position and maps points in camera space to world space."""

    n, num_rays, num_steps, channels = points.shape

    points, z_vals = perturb_points(points, z_vals, ray_directions)


    camera_origin, pitch, yaw = sample_camera_positions(n=points.shape[0], r=1, horizontal_stddev=h_stddev, vertical_stddev=v_stddev, horizontal_mean=h_mean, vertical_mean=v_mean, mode=mode)
    forward_vector = normalize_vecs(-camera_origin)

    cam2world_matrix = create_cam2world_matrix(forward_vector, camera_origin)

    points_homogeneous = torch.ones((points.shape[0], points.shape[1], points.shape[2], points.shape[3] + 1))
    points_homogeneous[:, :, :, :3] = points

    # should be n x 4 x 4 , n x r^2 x num_steps x 4
    transformed_points = torch.bmm(cam2world_matrix, points_homogeneous.reshape(n, -1, 4).permute(0,2,1)).permute(0, 2, 1).reshape(n, num_rays, num_steps, 4)


    transformed_ray_directions = torch.bmm(cam2world_matrix[..., :3, :3], ray_directions.reshape(n, -1, 3).permute(0,2,1)).permute(0, 2, 1).reshape(n, num_rays, 3)

    homogeneous_origins = torch.zeros((n, 4, num_rays))
    homogeneous_origins[:, 3, :] = 1
    transformed_ray_origins = torch.bmm(cam2world_matrix, homogeneous_origins).permute(0, 2, 1).reshape(n, num_rays, 4)[..., :3]

    return transformed_points[..., :3], z_vals, transformed_ray_directions, transformed_ray_origins, pitch, yaw


In [5]:
def perturb_points(points, z_vals, ray_directions):
    distance_between_points = z_vals[:,:,1:2,:] - z_vals[:,:,0:1,:]
    offset = (torch.rand(z_vals.shape)-0.5) * distance_between_points
    z_vals = z_vals + offset

    points = points + offset * ray_directions.unsqueeze(2)
    return points, z_vals

perturb_points(points,z_vals,ray_d_cam)

(tensor([[[[-0.5743, -0.5743, -2.1435],
           [-0.9381, -0.9381, -3.5009],
           [-1.1018, -1.1018, -4.1121],
           [-1.5789, -1.5789, -5.8925]],
 
          [[-0.4802, -0.6723, -2.5090],
           [-0.5245, -0.7344, -2.7407],
           [-0.8669, -1.2137, -4.5296],
           [-1.2100, -1.6940, -6.3219]],
 
          [[-0.2357, -0.5500, -2.0525],
           [-0.3483, -0.8128, -3.0333],
           [-0.4669, -1.0894, -4.0657],
           [-0.7161, -1.6709, -6.2360]],
 
          ...,
 
          [[ 0.1712,  0.3995, -1.4908],
           [ 0.3256,  0.7597, -2.8352],
           [ 0.4508,  1.0518, -3.9255],
           [ 0.7288,  1.7006, -6.3469]],
 
          [[ 0.3827,  0.5358, -1.9998],
           [ 0.6892,  0.9649, -3.6012],
           [ 0.8429,  1.1800, -4.4038],
           [ 1.1490,  1.6086, -6.0035]],
 
          [[ 0.4187,  0.4187, -1.5626],
           [ 0.7529,  0.7529, -2.8100],
           [ 1.0823,  1.0823, -4.0390],
           [ 1.5178,  1.5178, -5.6643]]],
 
 
  

In [6]:
z_vals.shape

torch.Size([3, 64, 4, 1])

In [7]:
import math
import numpy as np
z=-torch.ones_like(x)/np.tan((2 * math.pi * 30 / 360)/2)

def normalize_vecs(vectors: torch.Tensor) -> torch.Tensor:
    """
    Normalize vector lengths.
    """
    return vectors / (torch.norm(vectors, dim=-1, keepdim=True))

normalize_vecs(torch.stack([x, y, z], -1))

tensor([[-0.2506, -0.2506, -0.9351],
        [-0.1818, -0.2545, -0.9498],
        [-0.1102, -0.2572, -0.9600],
        [-0.0369, -0.2586, -0.9653],
        [ 0.0369, -0.2586, -0.9653],
        [ 0.1102, -0.2572, -0.9600],
        [ 0.1818, -0.2545, -0.9498],
        [ 0.2506, -0.2506, -0.9351],
        [-0.2545, -0.1818, -0.9498],
        [-0.1847, -0.1847, -0.9653],
        [-0.1121, -0.1868, -0.9760],
        [-0.0376, -0.1878, -0.9815],
        [ 0.0376, -0.1878, -0.9815],
        [ 0.1121, -0.1868, -0.9760],
        [ 0.1847, -0.1847, -0.9653],
        [ 0.2545, -0.1818, -0.9498],
        [-0.2572, -0.1102, -0.9600],
        [-0.1868, -0.1121, -0.9760],
        [-0.1134, -0.1134, -0.9871],
        [-0.0380, -0.1140, -0.9928],
        [ 0.0380, -0.1140, -0.9928],
        [ 0.1134, -0.1134, -0.9871],
        [ 0.1868, -0.1121, -0.9760],
        [ 0.2572, -0.1102, -0.9600],
        [-0.2586, -0.0369, -0.9653],
        [-0.1878, -0.0376, -0.9815],
        [-0.1140, -0.0380, -0.9928],
 

In [8]:
torch.stack([x,y,z],-1)

tensor([[-1.0000, -1.0000, -3.7321],
        [-0.7143, -1.0000, -3.7321],
        [-0.4286, -1.0000, -3.7321],
        [-0.1429, -1.0000, -3.7321],
        [ 0.1429, -1.0000, -3.7321],
        [ 0.4286, -1.0000, -3.7321],
        [ 0.7143, -1.0000, -3.7321],
        [ 1.0000, -1.0000, -3.7321],
        [-1.0000, -0.7143, -3.7321],
        [-0.7143, -0.7143, -3.7321],
        [-0.4286, -0.7143, -3.7321],
        [-0.1429, -0.7143, -3.7321],
        [ 0.1429, -0.7143, -3.7321],
        [ 0.4286, -0.7143, -3.7321],
        [ 0.7143, -0.7143, -3.7321],
        [ 1.0000, -0.7143, -3.7321],
        [-1.0000, -0.4286, -3.7321],
        [-0.7143, -0.4286, -3.7321],
        [-0.4286, -0.4286, -3.7321],
        [-0.1429, -0.4286, -3.7321],
        [ 0.1429, -0.4286, -3.7321],
        [ 0.4286, -0.4286, -3.7321],
        [ 0.7143, -0.4286, -3.7321],
        [ 1.0000, -0.4286, -3.7321],
        [-1.0000, -0.1429, -3.7321],
        [-0.7143, -0.1429, -3.7321],
        [-0.4286, -0.1429, -3.7321],
 

In [44]:
def get_initial_rays_trig(n, num_steps, fov, resolution, ray_start, ray_end):
    """Returns sample points, z_vals, and ray directions in camera space."""

    W, H = resolution
    # Create full screen NDC (-1 to +1) coords [x, y, 0, 1].
    # Y is flipped to follow image memory layouts.
    x, y = torch.meshgrid(torch.linspace(-1, 1, W),
                          torch.linspace(1, -1, H, )) # x,y grid , it descrete -1 to 1 with W resolution
    x = x.T.flatten() # transform하면 적절하게 x좌표에 맞는다.
    y = y.T.flatten() #  transform하면 잘 맞는다.
    z = -torch.ones_like(x) / np.tan((2 * math.pi * fov / 360)/2) # 

    rays_d_cam = normalize_vecs(torch.stack([x, y, z], -1))


    z_vals = torch.linspace(ray_start, ray_end, num_steps).reshape(1, num_steps, 1).repeat(W*H, 1, 1)
    points = rays_d_cam.unsqueeze(1).repeat(1, num_steps, 1) * z_vals

    points = torch.stack(n*[points])
    z_vals = torch.stack(n*[z_vals])
    rays_d_cam = torch.stack(n*[rays_d_cam])

    return points, z_vals, rays_d_cam

points,z_vals,rays_d_cam=get_initial_rays_trig(3,4,30,(8,8),2,6)


In [10]:
points.shape,z_vals.shape,rays_d_cam.shape

(torch.Size([1, 64, 1, 3]), torch.Size([1, 64, 1, 1]), torch.Size([1, 64, 3]))

In [11]:
x, y = torch.meshgrid(torch.linspace(-1, 1, 8, ),
                          torch.linspace(-1, 1, 8))
x,y

(tensor([[-1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000],
         [-0.7143, -0.7143, -0.7143, -0.7143, -0.7143, -0.7143, -0.7143, -0.7143],
         [-0.4286, -0.4286, -0.4286, -0.4286, -0.4286, -0.4286, -0.4286, -0.4286],
         [-0.1429, -0.1429, -0.1429, -0.1429, -0.1429, -0.1429, -0.1429, -0.1429],
         [ 0.1429,  0.1429,  0.1429,  0.1429,  0.1429,  0.1429,  0.1429,  0.1429],
         [ 0.4286,  0.4286,  0.4286,  0.4286,  0.4286,  0.4286,  0.4286,  0.4286],
         [ 0.7143,  0.7143,  0.7143,  0.7143,  0.7143,  0.7143,  0.7143,  0.7143],
         [ 1.0000,  1.0000,  1.0000,  1.0000,  1.0000,  1.0000,  1.0000,  1.0000]]),
 tensor([[-1.0000, -0.7143, -0.4286, -0.1429,  0.1429,  0.4286,  0.7143,  1.0000],
         [-1.0000, -0.7143, -0.4286, -0.1429,  0.1429,  0.4286,  0.7143,  1.0000],
         [-1.0000, -0.7143, -0.4286, -0.1429,  0.1429,  0.4286,  0.7143,  1.0000],
         [-1.0000, -0.7143, -0.4286, -0.1429,  0.1429,  0.4286,  0.7143,  1.0000],
  

In [12]:
x.T.flatten()

tensor([-1.0000, -0.7143, -0.4286, -0.1429,  0.1429,  0.4286,  0.7143,  1.0000,
        -1.0000, -0.7143, -0.4286, -0.1429,  0.1429,  0.4286,  0.7143,  1.0000,
        -1.0000, -0.7143, -0.4286, -0.1429,  0.1429,  0.4286,  0.7143,  1.0000,
        -1.0000, -0.7143, -0.4286, -0.1429,  0.1429,  0.4286,  0.7143,  1.0000,
        -1.0000, -0.7143, -0.4286, -0.1429,  0.1429,  0.4286,  0.7143,  1.0000,
        -1.0000, -0.7143, -0.4286, -0.1429,  0.1429,  0.4286,  0.7143,  1.0000,
        -1.0000, -0.7143, -0.4286, -0.1429,  0.1429,  0.4286,  0.7143,  1.0000,
        -1.0000, -0.7143, -0.4286, -0.1429,  0.1429,  0.4286,  0.7143,  1.0000])

In [13]:
z = -torch.ones_like(x) / np.tan((2 * math.pi * 30 / 360)/2)
z

tensor([[-3.7321, -3.7321, -3.7321, -3.7321, -3.7321, -3.7321, -3.7321, -3.7321],
        [-3.7321, -3.7321, -3.7321, -3.7321, -3.7321, -3.7321, -3.7321, -3.7321],
        [-3.7321, -3.7321, -3.7321, -3.7321, -3.7321, -3.7321, -3.7321, -3.7321],
        [-3.7321, -3.7321, -3.7321, -3.7321, -3.7321, -3.7321, -3.7321, -3.7321],
        [-3.7321, -3.7321, -3.7321, -3.7321, -3.7321, -3.7321, -3.7321, -3.7321],
        [-3.7321, -3.7321, -3.7321, -3.7321, -3.7321, -3.7321, -3.7321, -3.7321],
        [-3.7321, -3.7321, -3.7321, -3.7321, -3.7321, -3.7321, -3.7321, -3.7321],
        [-3.7321, -3.7321, -3.7321, -3.7321, -3.7321, -3.7321, -3.7321, -3.7321]])

In [14]:
z_vals=torch.linspace(2, 6, 4).reshape(1, 4, 1).repeat(64, 1, 1)
z_vals.shape

torch.Size([64, 4, 1])

In [15]:
ray_d_cam= normalize_vecs(torch.stack([x, y, z], -1))
print(ray_d_cam.shape)
print(ray_d_cam)

torch.Size([8, 8, 3])
tensor([[[-0.2506, -0.2506, -0.9351],
         [-0.2545, -0.1818, -0.9498],
         [-0.2572, -0.1102, -0.9600],
         [-0.2586, -0.0369, -0.9653],
         [-0.2586,  0.0369, -0.9653],
         [-0.2572,  0.1102, -0.9600],
         [-0.2545,  0.1818, -0.9498],
         [-0.2506,  0.2506, -0.9351]],

        [[-0.1818, -0.2545, -0.9498],
         [-0.1847, -0.1847, -0.9653],
         [-0.1868, -0.1121, -0.9760],
         [-0.1878, -0.0376, -0.9815],
         [-0.1878,  0.0376, -0.9815],
         [-0.1868,  0.1121, -0.9760],
         [-0.1847,  0.1847, -0.9653],
         [-0.1818,  0.2545, -0.9498]],

        [[-0.1102, -0.2572, -0.9600],
         [-0.1121, -0.1868, -0.9760],
         [-0.1134, -0.1134, -0.9871],
         [-0.1140, -0.0380, -0.9928],
         [-0.1140,  0.0380, -0.9928],
         [-0.1134,  0.1134, -0.9871],
         [-0.1121,  0.1868, -0.9760],
         [-0.1102,  0.2572, -0.9600]],

        [[-0.0369, -0.2586, -0.9653],
         [-0.0376, -0.

In [17]:
def normalize_vecs(vectors: torch.Tensor) -> torch.Tensor:
    """
    Normalize vector lengths.
    """
    return vectors / (torch.norm(vectors, dim=-1, keepdim=True))

x, y = torch.meshgrid(torch.linspace(-1, 1, 8, ),
                          torch.linspace(-1, 1, 8))
x=x.T.flatten()
y=y.T.flatten()
z = -torch.ones_like(x) / np.tan((2 * math.pi * 30 / 360)/2)

ray_d_cam= normalize_vecs(torch.stack([x, y, z], -1))
z_vals=torch.linspace(2, 6, 4).reshape(1, 4, 1).repeat(64, 1, 1)
points = ray_d_cam.unsqueeze(1).repeat(1, 4, 1) * z_vals
print(points.shape)
print(points)

torch.Size([64, 4, 3])
tensor([[[-0.5011, -0.5011, -1.8702],
         [-0.8352, -0.8352, -3.1170],
         [-1.1693, -1.1693, -4.3639],
         [-1.5034, -1.5034, -5.6107]],

        [[-0.3636, -0.5090, -1.8997],
         [-0.6060, -0.8484, -3.1661],
         [-0.8484, -1.1877, -4.4325],
         [-1.0907, -1.5270, -5.6990]],

        [[-0.2205, -0.5145, -1.9201],
         [-0.3675, -0.8575, -3.2001],
         [-0.5145, -1.2005, -4.4802],
         [-0.6615, -1.5434, -5.7602]],

        [[-0.0739, -0.5173, -1.9305],
         [-0.1232, -0.8621, -3.2176],
         [-0.1724, -1.2070, -4.5046],
         [-0.2217, -1.5519, -5.7916]],

        [[ 0.0739, -0.5173, -1.9305],
         [ 0.1232, -0.8621, -3.2176],
         [ 0.1724, -1.2070, -4.5046],
         [ 0.2217, -1.5519, -5.7916]],

        [[ 0.2205, -0.5145, -1.9201],
         [ 0.3675, -0.8575, -3.2001],
         [ 0.5145, -1.2005, -4.4802],
         [ 0.6615, -1.5434, -5.7602]],

        [[ 0.3636, -0.5090, -1.8997],
         [ 0.60

In [29]:
import random
def sample_camera_positions( n=1, r=1, horizontal_stddev=1, vertical_stddev=1, horizontal_mean=math.pi*0.5, vertical_mean=math.pi*0.5, mode='normal'):
    """
    Samples n random locations along a sphere of radius r. Uses the specified distribution.
    Theta is yaw in radians (-pi, pi)
    Phi is pitch in radians (0, pi)
    """

    if mode == 'uniform':
        theta = (torch.rand((n, 1)) - 0.5) * 2 * horizontal_stddev + horizontal_mean
        phi = (torch.rand((n, 1)) - 0.5) * 2 * vertical_stddev + vertical_mean
				

    elif mode == 'normal' or mode == 'gaussian':
        theta = torch.randn((n, 1)) * horizontal_stddev + horizontal_mean
        phi = torch.randn((n, 1)) * vertical_stddev + vertical_mean

    elif mode == 'hybrid':
        if random.random() < 0.5:
            theta = (torch.rand((n, 1)) - 0.5) * 2 * horizontal_stddev * 2 + horizontal_mean
            phi = (torch.rand((n, 1)) - 0.5) * 2 * vertical_stddev * 2 + vertical_mean
        else:
            theta = torch.randn((n, 1)) * horizontal_stddev + horizontal_mean
            phi = torch.randn((n, 1)) * vertical_stddev + vertical_mean

    #elif mode == 'truncated_gaussian':
    #    theta = truncated_normal_(torch.zeros((n, 1))) * horizontal_stddev + horizontal_mean
    #    phi = truncated_normal_(torch.zeros((n, 1))) * vertical_stddev + vertical_mean

    elif mode == 'spherical_uniform':
        theta = (torch.rand((n, 1)) - .5) * 2 * horizontal_stddev + horizontal_mean
        v_stddev, v_mean = vertical_stddev / math.pi, vertical_mean / math.pi
        v = ((torch.rand((n,1) ) - .5) * 2 * v_stddev + v_mean)
        v = torch.clamp(v, 1e-5, 1 - 1e-5)
        phi = torch.arccos(1 - 2 * v)

    else:
        # Just use the mean.
        theta = torch.ones((n, 1), dtype=torch.float) * horizontal_mean
        phi = torch.ones((n, 1),  dtype=torch.float) * vertical_mean

    phi = torch.clamp(phi, 1e-5, math.pi - 1e-5)

    output_points = torch.zeros((n, 3))
    output_points[:, 0:1] = r*torch.sin(phi) * torch.cos(theta)
    output_points[:, 2:3] = r*torch.sin(phi) * torch.sin(theta)
    output_points[:, 1:2] = r*torch.cos(phi)

    return output_points, phi, theta

In [30]:
camera_origin, pitch, yaw = sample_camera_positions(n=points.shape[0])

In [20]:
def create_cam2world_matrix(forward_vector , origin):
  """Takes in the direction the camera is pointing and the camera origin and returns a cam2world matrix."""
  forward_vector = normalize_vecs(forward_vector)
  up_vector = torch.tensor([0, 1, 0], dtype=torch.float).expand_as(forward_vector)

  left_vector = normalize_vecs(torch.cross(up_vector, forward_vector, dim=-1))

  up_vector = normalize_vecs(torch.cross(forward_vector, left_vector, dim=-1))

  rotation_matrix = torch.eye(4).unsqueeze(0).repeat(forward_vector.shape[0], 1, 1)
  rotation_matrix[:, :3, :3] = torch.stack((-left_vector, up_vector, -forward_vector), axis=-1)

  translation_matrix = torch.eye(4).unsqueeze(0).repeat(forward_vector.shape[0], 1, 1)
  translation_matrix[:, :3, 3] = origin

  cam2world = translation_matrix @ rotation_matrix

  return cam2world

In [31]:
forward_vector=normalize_vecs(-camera_origin)
cam2world_matrix=create_cam2world_matrix(forward_vector,camera_origin)

In [34]:
print(cam2world_matrix.shape)
print(cam2world_matrix)

torch.Size([64, 4, 4])
tensor([[[ 4.6786e-01,  8.1488e-01,  3.4218e-01,  3.4218e-01],
         [ 0.0000e+00,  3.8717e-01, -9.2201e-01, -9.2201e-01],
         [-8.8380e-01,  4.3137e-01,  1.8114e-01,  1.8114e-01],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  1.0000e+00]],

        [[ 8.5696e-01,  5.1538e-01, -5.1538e-06, -5.1538e-06],
         [ 0.0000e+00,  1.0000e-05,  1.0000e+00,  1.0000e+00],
         [ 5.1538e-01, -8.5696e-01,  8.5696e-06,  8.5696e-06],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  1.0000e+00]],

        [[ 9.8313e-01,  1.2247e-01, -1.3585e-01, -1.3585e-01],
         [ 0.0000e+00,  7.4272e-01,  6.6960e-01,  6.6960e-01],
         [ 1.8291e-01, -6.5831e-01,  7.3019e-01,  7.3019e-01],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  1.0000e+00]],

        ...,

        [[ 1.7106e-01,  9.5204e-01, -2.5368e-01, -2.5368e-01],
         [ 0.0000e+00,  2.5747e-01,  9.6629e-01,  9.6629e-01],
         [ 9.8526e-01, -1.6529e-01,  4.4042e-02,  4.4042e-02],
         [ 0

In [35]:
def transform_sampled_points(points, z_vals, ray_directions, h_stddev=1, v_stddev=1, h_mean=math.pi * 0.5, v_mean=math.pi * 0.5, mode='normal'):
    """Samples a camera position and maps points in camera space to world space."""

    n, num_rays, num_steps, channels = points.shape

    points, z_vals = perturb_points(points, z_vals, ray_directions)


    camera_origin, pitch, yaw = sample_camera_positions(n=points.shape[0], r=1, horizontal_stddev=h_stddev, vertical_stddev=v_stddev, horizontal_mean=h_mean, vertical_mean=v_mean, mode=mode)
    forward_vector = normalize_vecs(-camera_origin)

    cam2world_matrix = create_cam2world_matrix(forward_vector, camera_origin)

    points_homogeneous = torch.ones((points.shape[0], points.shape[1], points.shape[2], points.shape[3] + 1))
    points_homogeneous[:, :, :, :3] = points

    # should be n x 4 x 4 , n x r^2 x num_steps x 4
    transformed_points = torch.bmm(cam2world_matrix, points_homogeneous.reshape(n, -1, 4).permute(0,2,1)).permute(0, 2, 1).reshape(n, num_rays, num_steps, 4)


    transformed_ray_directions = torch.bmm(cam2world_matrix[..., :3, :3], ray_directions.reshape(n, -1, 3).permute(0,2,1)).permute(0, 2, 1).reshape(n, num_rays, 3)

    homogeneous_origins = torch.zeros((n, 4, num_rays))
    homogeneous_origins[:, 3, :] = 1
    transformed_ray_origins = torch.bmm(cam2world_matrix, homogeneous_origins).permute(0, 2, 1).reshape(n, num_rays, 4)[..., :3]

    return transformed_points[..., :3], z_vals, transformed_ray_directions, transformed_ray_origins, pitch, yaw

In [45]:
transformed_points, z_vals, transformed_ray_directions, transformed_ray_origins, pitch, yaw = transform_sampled_points(points,z_vals,rays_d_cam)

In [54]:
transformed_points.shape,z_vals.shape,transformed_ray_directions.shape,transformed_ray_origins.shape,pitch.shape,yaw.shape

(torch.Size([3, 64, 4, 3]),
 torch.Size([3, 64, 4, 1]),
 torch.Size([3, 64, 3]),
 torch.Size([3, 64, 3]),
 torch.Size([3, 1]),
 torch.Size([3, 1]))

In [63]:
transformed_ray_directions_expanded=torch.unsqueeze(transformed_ray_directions,-2).expand(-1,-1,4,-1).reshape(3,8*8*4,3)
transformed_points=transformed_points.reshape(3,8*8*4,3)
print(transformed_ray_directions_expanded.shape,transformed_points.shape)

torch.Size([3, 256, 3]) torch.Size([3, 256, 3])
